In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [ ]:
# 1. Сбор ссылок на новостные статьи

In [1]:
%%time

re_links=[]
for it in range(1,11):
    temp_url = "https://www.securitylab.ru/news/page1_" + str(it) + ".php"
    temp_page = rq.get(temp_url)
    temp_page.encoding = "UTF-8"
    soup=BeautifulSoup(temp_page.text, features = "html.parser")
    for link in soup.find_all("a"):
        for i in re.findall(r"news/\d+", str(link.get("href"))):
            if len(str(i)) > 1:
                re_links.append(str(i))
            else:
                pass
    time.sleep(2)
    
news_links=[]
for i in re_links:
    news_links.append("https://www.securitylab.ru/"+i+".php")

NameError: name 'rq' is not defined

In [ ]:
#2. Сбор заголовков, тегов и текстов

In [108]:
%%time

texts = []
titles = []
subtitles = []
tags = []

for link in news_links:
    temp_tags = []
    temp_url = link
    temp_page = rq.get(temp_url)
    temp_soup = BeautifulSoup(temp_page.text, "html.parser")
    titles.append(temp_soup.title.text.strip())
    subtitles.append(temp_soup.p.text.strip())
    for i in temp_soup.find_all("a", class_ = "tag tag-outline-primary"):
        temp_tags.append(i.text.strip())
    temp_tags=list(set(temp_tags))
    tags.append(tuple(temp_tags))
    temp_string=""
    temp_string += temp_soup.find("div", itemprop = "description").text.strip()
    texts.append(temp_string)
    time.sleep(2)

CPU times: user 1.66 s, sys: 47.7 ms, total: 1.71 s
Wall time: 27.4 s


In [ ]:
#3. Форматирование текста

In [109]:
clear_texts=[]

for i in texts:
    t=re.sub(r"(\r\n\s*\t)","",i)
    ct=re.sub(r"\s{2,10}|\r+|\n+|\t+"," ",t)
    clear_texts.append(ct)

In [ ]:
#4. Сбор датафрейма

In [110]:
data_list = titles,subtitles,tags,clear_texts

SecurityLab_df = pd.DataFrame(data_list, columns=news_links).T
SecurityLab_df = SecurityLab_df.rename(columns={0:"Title", 1:"Subtitle", 2:"Tags", 3:"Text"})
display(SecurityLab_df)

,Title,Subtitle,Tags,Text
https://www.securitylab.ru/news/536037.php,Неизвестное вредоносное ПО терроризирует игорн...,Новая китайская группировка использует интерпр...,"(Windows, C2, Китай, боковое перемещение, RAT,...",Исследователи кибербезопасности SentinelLabs с...
https://www.securitylab.ru/news/536036.php,Взломанный блокчейн-мост Wormhole: Хакер начин...,"Хакер, ответственный за крупный взлом блокчейн...","(кроссчейн-криптоплатформа, Wormhole, взлом, д...","Хакер, ответственный за взлом блокчейн-моста W..."
https://www.securitylab.ru/news/536034.php,Данные китайской военной разведки утекли в сеть,Секретные документы содержат планы разведки и ...,"(утечка данных, Китай, BreachForums, разведка,...","16 января китайские пользователи обнаружили, ч..."
https://www.securitylab.ru/news/536035.php,"Riot Games объявляет о крупной кибератаке, исх...","Riot Games заявила, что отложит выпуск игровых...","(кибератака, взлом, Teamfight Tactics, исходны...",Студия Riot Games опубликовала открытое письмо...
https://www.securitylab.ru/news/536033.php,Кремль объявил о желании вернуть в Россию квал...,Квалифицированные кадры представляют для Росси...,"(Песков, IT-специалисты, квалифицированные кад...","Для того, чтобы привлечь обратно в страну уеха..."
https://www.securitylab.ru/news/536032.php,Приглашаем принять участие в исследовании,Чем замещают и как поддерживают ИТ инфраструкт...,"(ИТ инфраструктура, исследование, M1Cloud, имп...","Мы M1Cloud проводим исследование, чтобы оценит..."
https://www.securitylab.ru/news/536031.php,Минцифры усилит контроль за ПО с открытым исхо...,Минцифры России рассматривает возможность введ...,"(требования регуляторов, ПО, реестр отечествен...",Минцифры России рассматривает возможность введ...
https://www.securitylab.ru/news/536030.php,Microsoft планирует пресечь на корню распростр...,Новая защитная опция будет добавлена в Microso...,"(XLM, Excel, Microsoft 365, полезная нагрузка,...",Microsoft работает над добавлением защиты надс...
https://www.securitylab.ru/news/536029.php,ChatGPT успешно сдал экзамен в медицинский вуз,Искусственный интеллект снова дал фору человеч...,"(OpenAI, нейросеть, ChatGPT, искусственный инт...","Ранее мы писали , что ChatGPT успешно сдал экз..."
https://www.securitylab.ru/news/536026.php,"Apple подтвердила, что недавно обнаруженная уя...",Речь идёт об уязвимости нулевого дня в старых ...,"(iPadOS, iOS, Apple, эксплойт, уязвимость нуле...","Уязвимость нулевого дня CVE-2022-42856, о кото..."


In [111]:
SecurityLab_df.to_csv('SecurityLab.csv', encoding='utf-8') 